Run scripts to create classifier from AFKABAN data.

In [1]:
import time
import os.path
import numpy as np
import pandas as pd
import pickle
import scipy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix
#from sklearn.neighbors import KNeighborsClassifier
import hyperopt
from hyperopt import tpe
from hpsklearn import HyperoptEstimator, k_neighbors_classifier, svc, lightgbm_classification, gaussian_nb, any_preprocessing,min_max_scaler, normalizer
import lightgbm
import hpsklearn
from datetime import timedelta, date
from tenacity import retry, stop_after_attempt
import AZKABANML
import sys, errno


# Read the dataframes

In [3]:
# Set paths
path = 'F:/AFKABAN/review2024/'
classifypath = 'F:/AFKABAN/Classify/'

df_120 = pd.read_feather(f'{path}/df_120_balanced.feather')
df_200 = pd.read_feather(f'{path}/df_200_balanced.feather')

df_120_sigbs = pd.read_feather(f'{path}/df_120_sigbs_balanced.feather')
df_200_sigbs = pd.read_feather(f'{path}/df_200_sigbs_balanced.feather')

track_120 = pd.read_feather(f'{path}/track_120_balanced.feather')
track_200 = pd.read_feather(f'{path}/track_200_balanced.feather')

# Run different iterations


## Preprocessing
Any! Give better results (+~0.2 on mean F1 score), either scale or min,max scaler or PCA.

In [4]:
# -- USER-DETERMINED PARAMETERS -----------------------------------------------
path = 'F:/AFKABAN/review2024/'
classifypath = 'F:/AFKABAN/Classify/'
d1 = date.today().strftime("%d-%m-%Y")

# NESTED CROSS-VALIDATION
#preprocessing = [min_max_scaler(unique_id)] # List of sklearn pre-processing modules
ex_preprocessing = [] # As above, see help(HyperoptEstimator) for info
n_splits = 10 # Value of k for k-fold cross-validation in outer loop
n_folds = 10 # Value of k for k-fold cross-validation in inner loop
max_evals = 50 # No. of HyperOpt trials
timeout = 300 # HyperOpt trial timeout (seconds)
n_jobs = -1 # No. of jobs to run in parallel; -1 uses all processors
retry_limit = 3 # No. of times to retry before failing

### kNN

In [5]:
# CLASSIFIER kNN 120
unique_id = 'kNN_'+d1+'_120' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify_group(df_120, clf, unique_id, classifypath, track_120, preprocessing=[])

Beginning fold 1 of 10
 67%|██████▋   | 2/3 [00:02<?, ?trial/s, best loss=?]
Beginning fold 2 of 10
 75%|███████▌  | 3/4 [00:00<?, ?trial/s, best loss=?]

In [4]:
# CLASSIFIER kNN 200
unique_id = 'kNN_'+d1+'_200' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify_group(df_200, clf, unique_id, classifypath, track_200, preprocessing=[])

Beginning fold 1 of 10
100%|██████████| 50/50 [00:02<00:00,  2.37s/trial, best loss: 0.01431307903366652]
Beginning fold 1 of 10
100%|██████████| 50/50 [00:04<00:00,  4.21s/trial, best loss: 0.01272277349029105]
Beginning fold 1 of 10
100%|██████████| 50/50 [00:02<00:00,  2.79s/trial, best loss: 0.013252820407926857]
Beginning fold 1 of 10
100%|██████████| 50/50 [00:02<00:00,  2.06s/trial, best loss: 0.013252942360463149]
Beginning fold 1 of 10
100%|██████████| 50/50 [00:03<00:00,  3.20s/trial, best loss: 0.013252820407926857]
Beginning fold 2 of 10
100%|██████████| 50/50 [00:02<00:00,  2.29s/trial, best loss: 0.009013785790031759]
Beginning fold 3 of 10
100%|██████████| 50/50 [00:04<00:00,  4.13s/trial, best loss: 0.007950702206443117]
Beginning fold 4 of 10
100%|██████████| 50/50 [00:02<00:00,  2.69s/trial, best loss: 0.012720139246221396]
Beginning fold 5 of 10
100%|██████████| 50/50 [00:02<00:00,  2.58s/trial, best loss: 0.01060032690649415]
Beginning fold 6 of 10
100%|██████████| 

In [17]:
# CLASSIFIER kNN 120
unique_id = 'kNN_'+d1+'_120_norm' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify_group(df_120, clf, unique_id, classifypath, track_120, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [00:02<00:00,  2.66s/trial, best loss: 0.029352220189054812]
Beginning fold 2 of 10
100%|██████████| 50/50 [00:02<00:00,  2.90s/trial, best loss: 0.03368614992026131]
Beginning fold 3 of 10
100%|██████████| 50/50 [00:03<00:00,  3.02s/trial, best loss: 0.03686654063749195]
Beginning fold 4 of 10
100%|██████████| 50/50 [00:02<00:00,  2.39s/trial, best loss: 0.03208731253695507]
Beginning fold 5 of 10
100%|██████████| 50/50 [00:03<00:00,  3.95s/trial, best loss: 0.03313393796208319]
Beginning fold 6 of 10
100%|██████████| 50/50 [00:02<00:00,  2.32s/trial, best loss: 0.029936783895862518]
Beginning fold 7 of 10
100%|██████████| 50/50 [00:03<00:00,  3.20s/trial, best loss: 0.0315260084169664]
Beginning fold 8 of 10
100%|██████████| 50/50 [00:03<00:00,  3.08s/trial, best loss: 0.034146485749176225]
Beginning fold 9 of 10
100%|██████████| 50/50 [00:02<00:00,  2.13s/trial, best loss: 0.031006928746433515]
Beginning fold 10 of 10
100%|██████████| 50/50 [00:02<00:00,  2.29

In [18]:
# CLASSIFIER kNN 200
unique_id = 'kNN_'+ d1 +'_200_norm' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify_group(df_200, clf, unique_id, classifypath, track_200, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [00:02<00:00,  2.59s/trial, best loss: 0.027065044834090335]
Beginning fold 2 of 10
100%|██████████| 50/50 [00:03<00:00,  3.03s/trial, best loss: 0.02873554085309582]
Beginning fold 3 of 10
100%|██████████| 50/50 [00:02<00:00,  2.58s/trial, best loss: 0.031364892227279006]
Beginning fold 4 of 10
100%|██████████| 50/50 [00:02<00:00,  2.29s/trial, best loss: 0.03137595191179943]
Beginning fold 5 of 10
100%|██████████| 50/50 [00:01<00:00,  1.98s/trial, best loss: 0.026548132445350703]
Beginning fold 6 of 10
100%|██████████| 50/50 [00:02<00:00,  2.36s/trial, best loss: 0.034505596740026956]
Beginning fold 7 of 10
100%|██████████| 50/50 [00:02<00:00,  2.88s/trial, best loss: 0.029198522298931606]
Beginning fold 8 of 10
100%|██████████| 50/50 [00:02<00:00,  2.35s/trial, best loss: 0.027588959591281936]
Beginning fold 9 of 10
100%|██████████| 50/50 [00:02<00:00,  2.11s/trial, best loss: 0.0302422783694547]
Beginning fold 10 of 10
100%|██████████| 50/50 [00:02<00:00,  2.

#### lightGBM

In [16]:
# CLASSIFIER lightGBM 120
unique_id = 'lightGBM_'+ d1 +'_120' # Unique ID for output file paths
clf = lightgbm.LGBMClassifier()#unique_id)  # Classifier
AZKABANML.main_classify_group(df_120, clf, unique_id, classifypath, track_120, preprocessing=[])

Beginning fold 1 of 10
100%|██████████| 50/50 [00:11<00:00, 11.33s/trial, best loss: 0.011724717860108091]
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001002 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8135
[LightGBM] [Info] Number of data points in the train set: 1876, number of used features: 32
[LightGBM] [Info] Start training from score -1.000206
[LightGBM] [Info] Start training from score -1.208960
[LightGBM] [Info] Start training from score -1.097547
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [17]:
# CLASSIFIER lightGBM 120
unique_id = 'lightGBM_'+ d1 +'_200' # Unique ID for output file paths
clf = lightgbm.LGBMClassifier()#unique_id)  # Classifier
AZKABANML.main_classify_group(df_200, clf, unique_id, classifypath, track_200, preprocessing=[])

Beginning fold 1 of 10
100%|██████████| 50/50 [00:06<00:00,  6.39s/trial, best loss: 0.005302470603830045]
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001007 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7573
[LightGBM] [Info] Number of data points in the train set: 1886, number of used features: 30
[LightGBM] [Info] Start training from score -1.021592
[LightGBM] [Info] Start training from score -1.203619
[LightGBM] [Info] Start training from score -1.079184
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

In [13]:
# CLASSIFIER lightGBM 120
unique_id = 'lightGBM_'+ d1 +'_120_norm' # Unique ID for output file paths
clf = lightgbm.LGBMClassifier()#unique_id)  # Classifier
AZKABANML.main_classify_group(df_120, clf, unique_id, classifypath, track_120, preprocessing=[normalizer(unique_id, norm='l2')])

KeyboardInterrupt: 

In [14]:
# CLASSIFIER lightGBM 200
unique_id = 'lightGBM_'+ d1 +'_200_norm' # Unique ID for output file paths
clf = lightgbm.LGBMClassifier()  # Classifier
AZKABANML.main_classify_group(df_200, clf, unique_id, classifypath, track_200, preprocessing=[normalizer(unique_id, norm='l2')])

Beginning fold 1 of 10
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [00:05<00:00,  5.91s/trial, best loss: 0.025026460588065347]
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000965 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7584
[LightGBM] [Info] Number of data points in the train set: 1886, number of used features: 30
[LightGBM] [Info] Start training from score -1.021592
[LightGBM] [Info] Start training from score -1.203619
[LightGBM] [Info] Start training from score -1.079184
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Beginning fold 2 of 10
100%|██████████| 50/50 [00:06<00:00,  6.00s/trial, best loss: 0.027194648509075492]
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000775 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7590
[LightGBM] [I

In [6]:
# CLASSIFIER SVC 120
unique_id = 'SVC_'+d1+'_120_norm' # Unique ID for output file paths
clf = svc(unique_id)  # Classifier
AZKABANML.main_classify_group(df_120, clf, unique_id, classifypath, track_120, preprocessing=[normalizer(unique_id, norm='l2')])

Resuming cross-validation from fold 11
Retraining model on full dataset
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [00:02<00:00,  2.89s/trial, best loss: 0.03375125145664293]


In [7]:
# CLASSIFIER SVC 200
unique_id = 'SVC_'+d1+'_200_norm' # Unique ID for output file paths
clf = svc(unique_id)  # Classifier
AZKABANML.main_classify_group(df_200, clf, unique_id, classifypath, track_200, preprocessing=[normalizer(unique_id, norm='l2')])

Resuming cross-validation from fold 11
Retraining model on full dataset
100%|██████████| 50/50 [00:03<00:00,  3.31s/trial, best loss: 0.03544227656742682]


## Linear
Balanced + PP

In [7]:
# CLASSIFIER kNN 120
unique_id = 'kNN_'+ d1 +'_120_sigbs' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier
AZKABANML.main_classify_group(df_120_sigbs, clf, unique_id, classifypath, track_120, preprocessing=[min_max_scaler(unique_id)])

Beginning fold 1 of 10
  0%|          | 0/1 [00:12<?, ?trial/s, best loss=?]
Beginning fold 1 of 10
100%|██████████| 50/50 [00:02<00:00,  2.29s/trial, best loss: 0.012253445774970673]
Beginning fold 2 of 10
100%|██████████| 50/50 [00:02<00:00,  2.55s/trial, best loss: 0.007465341005164494]
Beginning fold 3 of 10
100%|██████████| 50/50 [00:02<00:00,  2.64s/trial, best loss: 0.007991024329073504]
Beginning fold 4 of 10
100%|██████████| 50/50 [00:02<00:00,  2.51s/trial, best loss: 0.0074655846303421924]
Beginning fold 5 of 10
100%|██████████| 50/50 [00:02<00:00,  2.60s/trial, best loss: 0.009058835472087967]
Beginning fold 6 of 10
100%|██████████| 50/50 [00:03<00:00,  3.10s/trial, best loss: 0.00852607020111873]
Beginning fold 7 of 10
100%|██████████| 50/50 [00:02<00:00,  2.64s/trial, best loss: 0.007994396407106752]
Beginning fold 8 of 10
100%|██████████| 50/50 [00:02<00:00,  2.47s/trial, best loss: 0.0058609413599670646]
Beginning fold 9 of 10
100%|██████████| 50/50 [00:02<00:00,  2.70s

In [8]:
# CLASSIFIER kNN 200
unique_id = 'kNN_'+ d1 +'_200_sigbs' # Unique ID for output file paths
clf = k_neighbors_classifier(unique_id)  # Classifier

AZKABANML.main_classify_group(df_200_sigbs, clf, unique_id, classifypath, track_200, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
100%|██████████| 50/50 [00:02<00:00,  2.57s/trial, best loss: 0.008481926606705659]
Beginning fold 2 of 10
100%|██████████| 50/50 [00:02<00:00,  2.52s/trial, best loss: 0.01272596698098638]
Beginning fold 3 of 10
100%|██████████| 50/50 [00:02<00:00,  2.58s/trial, best loss: 0.010072019584583036]
Beginning fold 4 of 10
100%|██████████| 50/50 [00:02<00:00,  2.48s/trial, best loss: 0.009009672972119964]
Beginning fold 5 of 10
100%|██████████| 50/50 [00:03<00:00,  3.16s/trial, best loss: 0.010072071069645916]
Beginning fold 6 of 10
100%|██████████| 50/50 [00:02<00:00,  2.50s/trial, best loss: 0.011129587458175494]
Beginning fold 7 of 10
100%|██████████| 50/50 [00:02<00:00,  2.47s/trial, best loss: 0.008474917904400026]
Beginning fold 8 of 10
100%|██████████| 50/50 [00:02<00:00,  2.60s/trial, best loss: 0.008471192936680194]
Beginning fold 9 of 10
100%|██████████| 50/50 [00:02<00:00,  2.34s/trial, best loss: 0.0074196069517669105]
Beginning fold 10 of 10
100%|████████

In [11]:
# CLASSIFIER lightGBM 120
unique_id = 'lightGBM_'+ d1 +'_120_sigbs' # Unique ID for output file paths
clf = lightgbm.LGBMClassifier()  # Classifier
AZKABANML.main_classify_group(df_120_sigbs, clf, unique_id, classifypath, track_120, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [00:06<00:00,  6.17s/trial, best loss: 0.00586422173921719]
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000979 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8152
[LightGBM] [Info] Number of data points in the train set: 1876, number of used features: 32
[LightGBM] [Info] Start training from score -1.000206
[LightGBM] [Info] Start training from score -1.208960
[LightGBM] [Info] Start training from score -1.097547
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warni

In [13]:
# CLASSIFIER lightGBM 200
unique_id = 'lightGBM_'+ d1 +'_200_sigbs' # Unique ID for output file paths
clf = lightgbm.LGBMClassifier()  # Classifier
AZKABANML.main_classify_group(df_200_sigbs, clf, unique_id, classifypath, track_200, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
  0%|          | 0/1 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 50/50 [00:19<00:00, 19.90s/trial, best loss: 0.0047721111464920085]
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002226 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7573
[LightGBM] [Info] Number of data points in the train set: 1886, number of used features: 30
[LightGBM] [Info] Start training from score -1.021592
[LightGBM] [Info] Start training from score -1.203619
[LightGBM] [Info] Start training from score -1.079184
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [War

In [14]:
# CLASSIFIER SVC 120
unique_id = 'SVC_'+ d1 +'_120_sigbs' # Unique ID for output file paths
clf = svc(unique_id)  # Classifier
AZKABANML.main_classify_group(df_120_sigbs, clf, unique_id, classifypath, track_120, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
100%|██████████| 50/50 [00:03<00:00,  3.32s/trial, best loss: 0.04697556563797978]
Beginning fold 2 of 10
100%|██████████| 50/50 [00:02<00:00,  2.48s/trial, best loss: 0.037399867740208426]
Beginning fold 3 of 10
100%|██████████| 50/50 [00:02<00:00,  2.57s/trial, best loss: 0.051288659925373015]
Beginning fold 4 of 10
 96%|█████████▋| 26/27 [00:00<?, ?trial/s, best loss=?]

job exception: The 'C' parameter of SVC must be a float in the range (0.0, inf). Got -0.15277319699281722 instead.



 96%|█████████▋| 26/27 [00:02<?, ?trial/s, best loss=?]
Resuming cross-validation from fold 4
Beginning fold 4 of 10
100%|██████████| 50/50 [00:03<00:00,  3.58s/trial, best loss: 0.014395257480954937]
Beginning fold 5 of 10
100%|██████████| 50/50 [00:02<00:00,  2.48s/trial, best loss: 0.010656004578311395]
Beginning fold 6 of 10
100%|██████████| 50/50 [00:03<00:00,  3.74s/trial, best loss: 0.04654724753356121]
Beginning fold 7 of 10
100%|██████████| 50/50 [00:02<00:00,  2.78s/trial, best loss: 0.05241840190494895]
Beginning fold 8 of 10
100%|██████████| 50/50 [00:02<00:00,  2.60s/trial, best loss: 0.045961931360429475]
Beginning fold 9 of 10
100%|██████████| 50/50 [00:02<00:00,  2.88s/trial, best loss: 0.012787409360327628]
Beginning fold 10 of 10
100%|██████████| 50/50 [00:02<00:00,  2.75s/trial, best loss: 0.04812662296692549]
Retraining model on full dataset
100%|██████████| 50/50 [00:04<00:00,  4.15s/trial, best loss: 0.04280329105139591]


In [15]:
# CLASSIFIER SVC 120
unique_id = 'SVC_'+ d1 +'_200_sigbs' # Unique ID for output file paths
clf = svc(unique_id)  # Classifier
AZKABANML.main_classify_group(df_200_sigbs, clf, unique_id, classifypath, track_200, preprocessing=any_preprocessing(unique_id))

Beginning fold 1 of 10
100%|██████████| 50/50 [00:02<00:00,  2.59s/trial, best loss: 0.02120462192291972]
Beginning fold 2 of 10
100%|██████████| 50/50 [00:02<00:00,  2.62s/trial, best loss: 0.12926884676555606]
Beginning fold 3 of 10
100%|██████████| 50/50 [00:03<00:00,  3.61s/trial, best loss: 0.030257948153690428]
Beginning fold 4 of 10
100%|██████████| 50/50 [00:02<00:00,  2.96s/trial, best loss: 0.024389300359519805]
Beginning fold 5 of 10
100%|██████████| 50/50 [00:03<00:00,  3.14s/trial, best loss: 0.09930612654625992]
Beginning fold 6 of 10
100%|██████████| 50/50 [00:02<00:00,  2.84s/trial, best loss: 0.09732478459285443]
Beginning fold 7 of 10
100%|██████████| 50/50 [00:03<00:00,  3.38s/trial, best loss: 0.1162454532645062]
Beginning fold 8 of 10
 97%|█████████▋| 31/32 [04:05<?, ?trial/s, best loss=?]
Beginning fold 9 of 10
100%|██████████| 50/50 [00:04<00:00,  4.18s/trial, best loss: 0.018547980061972558]
Beginning fold 10 of 10
100%|██████████| 50/50 [00:04<00:00,  4.56s/tri